# Run a scripts to fine-tune a llama-2 model on generating python code

In [ ]:
!pip install -r requirements.txt

## Run the fine-tuning script

In [ ]:
!python train.py --model-name NousResearch/Llama-2-7b-hf --hf_repo edumunozsala/llama-2-7b-int4-python-18k-alpaca --trained-model-name llama-2-7b-int4-python-18k-alpaca --split train --epochs 3 --bf16

In [ ]:
!python --version

## Load the model trained from the hub and test it

In [ ]:
from random import randrange

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "edumunozsala/llama-2-7b-int4-python-18k-alpaca"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, torch_dtype=torch.float16,
                                             device_map=device_map).to("cuda")

instruction="Develop a Python program that prints Hello, World! whenever it is run"
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")

